# Time+Day Basket Analysis



In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt

from matplotlib.font_manager import FontProperties
import seaborn as sns; sns.set()
from itertools import combinations, combinations_with_replacement
import pandas as pd
import numpy as np
pd.set_option("display.max_columns",999)

import seaborn as sns; sns.set()

from impala.dbapi import connect
from impala.util import as_pandas

In [2]:
def query_cluster(query: str):
    host = "server.name.de"
    port = 12345
    cursor = connect(host=host, port=port).cursor()
    cursor.execute(query)
    return as_pandas(cursor)

In [3]:
##############################
# 
# S E T   T H E    B U  ! ! !
#
##############################
##############################
# medium
# mail OR newsletter
##############################
Marke = 'xxx'
BU = 'www.xxx.de'

In [4]:
query = f'''
select fullvisitorid, visitid, visitstarttime, productprice, medium
from  attribution.tableName
where hostname = '{BU}'
and actiontypetext='zum_warenkorb'
'''

data = query_cluster(query) # returns pandas dataframe

In [5]:
###################################################
# Pickle!
###################################################
# data.to_pickle('file.pickle')
# data = pd.read_pickle('file_old.pickle')

In [6]:
data.shape

(597410, 5)

In [7]:
data['timestamp'] = pd.to_datetime(data['visitstarttime'], unit='s')

In [8]:
data['weekDay'] = data.timestamp.dt.weekday_name
data['hour'] = data.timestamp.dt.hour

In [9]:
data.head()

,fullvisitorid,visitid,visitstarttime,productprice,medium,timestamp,weekDay,hour
0,62419865028560645,1561470912,1561470912,2299,cpc,2019-06-25 13:55:12,Tuesday,13
1,62419865028560645,1561470912,1561470912,2299,cpc,2019-06-25 13:55:12,Tuesday,13
2,876310760299576265,1561470989,1561470989,4619,cpc,2019-06-25 13:56:29,Tuesday,13
3,876310760299576265,1561470989,1561470989,10919,cpc,2019-06-25 13:56:29,Tuesday,13
4,370703817199724574,1561471006,1561471006,6995,cpc,2019-06-25 13:56:46,Tuesday,13


# Alle Kanäle

In [10]:
data_wd_h = data.groupby(['weekDay','hour'])['visitid'].count()

In [11]:
df1 = data.groupby(['weekDay','hour']).agg({'visitid': ['count'],'productprice': ['sum']})

In [12]:
df3_id = df1.visitid.unstack().loc[:,'count']
#df3_pprice = df2.pprice.unstack().loc[:,'sum']

In [13]:
df4_id = df3_id.loc[['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday',],:]
#df4_pprice = df3_pprice.loc[['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday',],:]

In [14]:
df5_id = df4_id.apply(lambda x:x/x.sum(),axis=1)
#df5_pprice = df4_pprice.apply(lambda x:x/x.sum(),axis=1)

In [15]:
df4_id

hour,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
weekDay,,,,,,,,,,,,,,,,,,,,,,,,
Monday,277,166,163,242,584,1375,2462,3894,6826,7145,6328,6375,5917,5872,6349,6399,6248,6342,5532,4526,3272,1889,1017,642
Tuesday,325,177,221,695,725,1533,2208,3448,4400,4785,4569,5015,5471,4974,5718,6602,5995,5795,5615,4435,3258,1966,1117,584
Wednesday,299,187,121,292,732,1392,2792,3732,5623,6158,5770,5336,5468,5621,5992,6474,6120,5729,5314,4529,3486,2303,1341,662
Thursday,340,209,170,364,944,1571,2425,3897,5323,5433,4802,4780,5186,5338,5323,6010,5555,5307,4995,4218,3276,1943,1032,575
Friday,224,137,253,350,634,1370,2642,3062,3864,3950,4063,4315,4694,5166,5761,5828,5014,4759,4612,4032,3007,1849,1176,680
Saturday,291,217,190,321,536,951,2029,2842,6063,6529,6429,6669,6702,7097,7152,7146,7036,6548,5661,4217,3687,2228,1745,744
Sunday,283,328,159,224,554,1149,2457,4648,6499,6938,6405,6242,6250,7406,7332,8095,7407,6978,6294,5308,3987,2323,1075,635


In [16]:
df4_id.sum(axis=1).to_frame()

,0
weekDay,
Monday,89842
Tuesday,79631
Wednesday,85473
Thursday,79016
Friday,71442
Saturday,93030
Sunday,98976


In [17]:
df4_id.sum().sum()

597410

In [2]:
fig, axs = plt.subplots(nrows=2, ncols=2, gridspec_kw = {'width_ratios':[1, 15]},figsize=(12,6))

sns.heatmap(df4_id.sum(axis=1).to_frame()/df4_id.sum().sum()*100,cmap="binary_r",linewidths=1,linecolor='grey',annot=True,fmt=".2f",annot_kws={"size": 8},cbar=False,ax=axs[1,0])
axs[1,0].set_title("Taglicher % der\n\'add\'-Ereignisse")

# hours
sns.heatmap(df4_id.sum().to_frame().T/df4_id.sum().sum()*100,cmap="binary_r",linewidths=1,linecolor='grey',annot=True,fmt=".2f",annot_kws={"size": 7},cbar=False,ax=axs[0,1])
axs[0,1].set_title("Stündlicher % der\n\'add\'-Ereignisse")

# mainplot
sns.heatmap(df5_id*100,cmap="binary_r",fmt=".2f",annot_kws={"size": 8},annot=True,cbar=False, ax=axs[1,1])#, fmt="f",size=3)
axs[1,1].set_title('Prozent der \'add\'-Ereignisse im Laufe des Tages und der Woche\n\
Quelle: '+Marke+' alle Kanäle \'add\'-Ereignisse seit dem 2019-11-18 aus yyy')

# remove the missing fourth plot
fig.delaxes(axs[0,0])

plt.tight_layout()
plt.savefig(BU+'/UhrzeitAnalyse_Prozent_AlleKanaele_'+Marke+'.png')
plt.show()

![title](img/prima.png)

In [4]:
fig, axs = plt.subplots(nrows=2, ncols=2, gridspec_kw = {'width_ratios':[1, 15]},figsize=(12,5))

sns.heatmap(df4_id.sum(axis=1).to_frame(),cmap="binary_r",linewidths=1,linecolor='grey',annot=True,fmt=".0f",annot_kws={"size": 8},cbar=False,ax=axs[1,0])
axs[1,0].set_title("Taglicher Anzahl der \n\'add\'-Ereignisse")

# hours
sns.heatmap(df4_id.sum().to_frame().T,cmap="binary_r",linewidths=1,linecolor='grey',annot=True,fmt=".0f",annot_kws={"size": 7},cbar=False,ax=axs[0,1])
axs[0,1].set_title("Stündlicher Anzahl der\n\'add\'-Ereignisse")

# mainplot
sns.heatmap(df4_id,cmap="binary_r",fmt=".0f",annot_kws={"size": 7},annot=True,cbar=False, ax=axs[1,1])#, fmt="f",size=3)
axs[1,1].set_title('Anzahl der \'add\'-Ereignisse im Laufe des Tages und der Woche\n\
Quelle: '+Marke+' alle Kanäle \'add\'-Ereignisse seit dem 2019-11-18 aus yyy')

# remove the missing fourth plot
fig.delaxes(axs[0,0])

plt.tight_layout()
plt.savefig(BU+'/UhrzeitAnalyse_Roh-Daten_AlleKanaele_'+Marke+'.png')
plt.show()

![title](img/seconda.png)

# Nur NL Daten

In [20]:
data_nl = data[(data.medium=='mail') | (data.medium=='newsletter')]

In [21]:
df1_nl = data_nl.groupby(['weekDay','hour']).agg({'visitid': ['count'],'productprice': ['sum']})

In [22]:
df1_nl.head()

visitid                                       productprice
               count                                                sum
weekDay hour                                                           
Friday  0         31  4999499922991499229918992999299917599349935991...
        1          2                                           65998995
        2         39  8999899969994599459949998499599999992495499989...
        3         23  6499649964992299559945996495699575994499299969...
        4         53  0379959995999849959996499899959997499749944994...

In [23]:
df3_nl_id = df1_nl.visitid.unstack().loc[:,'count']

In [24]:
df4_nl_id = df3_nl_id.loc[['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday',],:]
df5_nl_id = df4_nl_id.apply(lambda x:x/x.sum(),axis=1)
# df5_nl_id.compute()

In [5]:
fig, axs = plt.subplots(nrows=2, ncols=2, gridspec_kw = {'width_ratios':[1, 15]},figsize=(12,5))

sns.heatmap(df4_nl_id.sum(axis=1).to_frame()/df4_nl_id.sum().sum()*100,cmap="binary_r",linewidths=1,linecolor='grey',annot=True,fmt=".2f",annot_kws={"size": 8},cbar=False,ax=axs[1,0])
axs[1,0].set_title("Taglicher % der\n\'add\'-Ereignisse")

# hours
sns.heatmap(df4_nl_id.sum().to_frame().T/df4_nl_id.sum().sum()*100,cmap="binary_r",linewidths=1,linecolor='grey',annot=True,fmt=".2f",annot_kws={"size": 7},cbar=False,ax=axs[0,1])
axs[0,1].set_title("Stündlicher % der\n\'add\'-Ereignisse")

# mainplot
sns.heatmap(df5_nl_id*100,cmap="binary_r",fmt=".2f",annot_kws={"size": 8},annot=True,cbar=False, ax=axs[1,1])#, fmt="f",size=3)
axs[1,1].set_title('Prozent der \'add\'-Ereignisse im Laufe des Tages und der Woche\n\
Quelle: '+Marke+' nur NL \'add\'-Ereignisse seit dem 22019-05-08 aus yyy')

# remove the missing fourth plot
fig.delaxes(axs[0,0])

plt.tight_layout()
plt.savefig(BU+'/UhrzeitAnalyse_Prozent_NurNL_'+Marke+'.png')
plt.show()

![title](img/terza.png)

In [6]:
fig, axs = plt.subplots(nrows=2, ncols=2, gridspec_kw = {'width_ratios':[1, 15]},figsize=(12,5))

# weekday
sns.heatmap(df4_nl_id.sum(axis=1).to_frame(),cmap="binary_r",linewidths=1,linecolor='grey',annot=True,fmt=".0f",annot_kws={"size": 8},cbar=False,ax=axs[1,0])
axs[1,0].set_title("Taglicher Anzahl der \n\'add\'-Ereignisse")

# hours
sns.heatmap(df4_nl_id.sum().to_frame().T,cmap="binary_r",linewidths=1,linecolor='grey',annot=True,fmt=".0f",annot_kws={"size": 7},cbar=False,ax=axs[0,1])
axs[0,1].set_title("Stündlicher Anzahl der\n\'add\'-Ereignisse")

# mainplot
sns.heatmap(df4_nl_id,cmap="binary_r",fmt=".0f",annot_kws={"size": 7},annot=True,cbar=False, ax=axs[1,1])#, fmt="f",size=3)
axs[1,1].set_title('Anzahl der \'add\'-Ereignisse im Laufe des Tages und der Woche\n\
Quelle: '+Marke+' nur NL \'add\'-Ereignisse seit dem 2019-05-08 aus yyy')

# remove the missing fourth plot
fig.delaxes(axs[0,0])

plt.tight_layout()
plt.savefig(BU+'/UhrzeitAnalyse_Roh-Daten_NurNL_'+Marke+'.png')
plt.show()

![title](img/quarta.png)

# Nicht NL-Daten

I removed the images here!

In [27]:
selection = (data.medium=='mail') | (data.medium=='newsletter')
data_Nnl = data[~selection]

In [28]:
df1_Nnl = data_Nnl.groupby(['weekDay','hour']).agg({'visitid': ['count'],'productprice': ['sum']})

In [29]:
df1_Nnl.head()

visitid                                       productprice
               count                                                sum
weekDay hour                                                           
Friday  0        193  4999499979996999599959995999799917999559955999...
        1        135  4599229939994999499949992299911999109997599119...
        2        214  2999299979997493999399949996999659965991099589...
        3        327  2999299935995599659945995599559965992999229922...
        4        581  3599489948991499299927998491599559914991499149...

In [30]:
df3_Nnl_id = df1_Nnl.visitid.unstack().loc[:,'count']

In [31]:
df4_Nnl_id = df3_Nnl_id.loc[['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday',],:]
df5_Nnl_id = df4_Nnl_id.apply(lambda x:x/x.sum(),axis=1)
# df5_nl_id.compute()

In [7]:
fig, axs = plt.subplots(nrows=2, ncols=2, gridspec_kw = {'width_ratios':[1, 15]},figsize=(12,5))
#fig.figure(figsize=(10,10))

# axs[0].plot(x1, y1, 'o-')

# weekday
sns.heatmap(df4_Nnl_id.sum(axis=1).to_frame()/df4_Nnl_id.sum().sum()*100,cmap="binary_r",linewidths=1,linecolor='grey',annot=True,fmt=".2f",annot_kws={"size": 8},cbar=False,ax=axs[1,0])
axs[1,0].set_title("Taglicher % der\n\'add\'-Ereignisse")

# hours
sns.heatmap(df4_Nnl_id.sum().to_frame().T/df4_Nnl_id.sum().sum()*100,cmap="binary_r",linewidths=1,linecolor='grey',annot=True,fmt=".2f",annot_kws={"size": 7},cbar=False,ax=axs[0,1])
axs[0,1].set_title("Stündlicher % der\n\'add\'-Ereignisse")

# mainplot
sns.heatmap(df5_Nnl_id*100,cmap="binary_r",fmt=".2f",annot_kws={"size": 8},annot=True,cbar=False, ax=axs[1,1])#, fmt="f",size=3)
axs[1,1].set_title('Prozent der \'add\'-Ereignisse im Laufe des Tages und der Woche\n\
Quelle: '+Marke+' nicht NL \'add\'-Ereignisse seit dem 2019-05-08 aus yyy')

# remove the missing fourth plot
fig.delaxes(axs[0,0])

plt.tight_layout()
plt.savefig(BU+'/UhrzeitAnalyse_Prozent_NichNL_'+Marke+'.png')
plt.show()

In [8]:
fig, axs = plt.subplots(nrows=2, ncols=2, gridspec_kw = {'width_ratios':[1, 15]},figsize=(12,5))
#fig.figure(figsize=(10,10))

# axs[0].plot(x1, y1, 'o-')

# weekday
sns.heatmap(df4_Nnl_id.sum(axis=1).to_frame(),cmap="binary_r",linewidths=1,linecolor='grey',annot=True,fmt=".0f",annot_kws={"size": 8},cbar=False,ax=axs[1,0])
axs[1,0].set_title("Taglicher Anzahl der \n\'add\'-Ereignisse")

# hours
sns.heatmap(df4_Nnl_id.sum().to_frame().T,cmap="binary_r",linewidths=1,linecolor='grey',annot=True,fmt=".0f",annot_kws={"size": 7},cbar=False,ax=axs[0,1])
axs[0,1].set_title("Stündlicher Anzahl der\n\'add\'-Ereignisse")

# mainplot
sns.heatmap(df4_Nnl_id,cmap="binary_r",fmt=".0f",annot_kws={"size": 7},annot=True,cbar=False, ax=axs[1,1])#, fmt="f",size=3)
axs[1,1].set_title('Anzahl der \'add\'-Ereignisse im Laufe des Tages und der Woche\n\
Quelle: '+Marke+' nicht NL \'add\'-Ereignisse seit dem 2019-05-08 aus yyy')

# remove the missing fourth plot
fig.delaxes(axs[0,0])

plt.tight_layout()
plt.savefig(BU+'/UhrzeitAnalyse_Roh-Daten_nichtNL_'+Marke+'.png')
plt.show()

## Exportiere Alles

In [34]:
df4_nl_id.to_csv(BU+'/Uhrzeitanalyse_'+Marke+'_nurNL.csv',decimal=',',sep=';')
df4_id.to_csv(BU+'/Uhrzeitanalyse_'+Marke+'_Gesamt.csv',decimal=',',sep=';')
df4_Nnl_id.to_csv(BU+'/Uhrzeitanalyse_'+Marke+'_ohneNL.csv',decimal=',',sep=';')